In [1]:
%run ./utils/commonImports.py
%run ./utils/tradingImports.py
%matplotlib inline

# Load Data

In [2]:
def load_and_divide(db, table, from_date=None, to_date=None, train_ratio=0.5, period=None):
    data = load_trading_data(db, table, from_date, to_date, period)
    data_train, data_test = divide_train_and_test(data, train_ratio)
    return data, data_train, data_test


db = 'D:\\Dropbox\\My work\\krypl-project\\sqlite\\ploniex-chart-data\\USDT_BTC.db'
data, data_train, data_test = load_and_divide(db, 'chart_data', from_date='2017-01-01', period='5min')
ohlc, ohlc_train, ohlc_test = data.copy(), data_train.copy(), data_test.copy()


# features = ['red', 'yellow', 'green', 'blue']
# data_manager_train = CurrencyDataManager(data_train['close'], data_train[features])
# data_manager_test = CurrencyDataManager(data_test['close'], data_test[features])
# wallet = {'usdt': 1000}
# contract_pair = ContractPair.new('usdt', 'btc')

# Price Movement Within a Day

In [56]:
import time

maxs = []
till_maxs = []
mins = []
df = ohlc.copy()
DAY = 1440

def _max_row(day_df, col):
    i = day_df[col].idxmax()
    max_row = day_df.iloc[i, :]
    return max_row

def _min_row(day_df, col):
    i = day_df[col].idxmin()
    min_row = day_df.iloc[i, :]
    return min_row

def _max(day_df):
    close_max_row = _max_row(day_df, 'close')
    open_max_row = _max_row(day_df, 'open')
    return (close_max_row, 'close') if close_max_row['close'] > open_max_row['open'] else (open_max_row, 'open')

def _return(v1, v2):
    return (v2 / v1) - 1


start_time = time.time()
for i in range(df.shape[0] - 1):
    if i % 1000 == 0:
        print(i, '%.3f s' % (time.time() - start_time))
        start_time = time.time()
    
    day_i = i + DAY if i + DAY < df.shape[0] else df.shape[0]-1
    row = df.iloc[i, :]
    day_df = df.iloc[i+1:day_i, :].reset_index().drop('index', axis=1).reset_index()
    max_row, max_col = _max(day_df)
    
    max_index = max_row['index'] if max_row['index'] > 0 else 1
    day_df = day_df.iloc[:max_index, :]
    min_row = _min_row(day_df, 'low')
    
    maxs.append(_return(row['close'], max_row[max_col]))
    till_maxs.append(max_row['index'])
    mins.append(_return(row['close'], min_row['low']))    

0 0.000 s
1000 4.900 s
2000 5.375 s
3000 4.804 s
4000 5.612 s
5000 4.998 s
6000 5.009 s
7000 4.621 s
8000 4.699 s
9000 4.645 s
10000 4.666 s
11000 4.909 s
12000 4.673 s
13000 4.649 s
14000 4.661 s
15000 4.608 s
16000 5.065 s
17000 4.644 s
18000 4.645 s
19000 4.708 s
20000 5.807 s
21000 5.637 s
22000 4.854 s
23000 5.050 s
24000 4.938 s
25000 4.782 s
26000 4.787 s
27000 4.766 s
28000 4.762 s
29000 4.862 s
30000 4.757 s
31000 4.857 s
32000 4.764 s
33000 4.752 s
34000 4.749 s
35000 4.831 s
36000 4.760 s
37000 4.860 s
38000 4.794 s
39000 4.803 s
40000 4.798 s
41000 4.867 s
42000 4.775 s
43000 4.831 s
44000 4.847 s
45000 4.781 s
46000 4.821 s
47000 4.797 s
48000 4.785 s
49000 4.831 s
50000 4.771 s
51000 4.815 s
52000 4.817 s
53000 4.757 s
54000 4.873 s
55000 4.781 s
56000 4.814 s
57000 4.790 s
58000 4.784 s
59000 4.753 s
60000 4.815 s
61000 4.789 s
62000 4.885 s
63000 4.781 s
64000 4.847 s
65000 4.751 s
66000 4.739 s
67000 4.752 s
68000 4.851 s
69000 4.796 s
70000 4.870 s
71000 5.257 s
72000

ValueError: attempt to get argmax of an empty sequence

In [60]:
maxs += [-1, -1]
till_maxs += [-1, -1]
mins += [-1, -1]

In [70]:
df = ohlc.copy()
df['maxReturn'] = maxs
df['tillMax'] = till_maxs
df['minReturn'] = mins

In [ ]:
df.write_csv('D:\\Dropbox\\My work\\krypl-project\\analysis\\price-movement-within-day.tsv', sep='\t', index=False)

In [63]:
df = df[['date', 'maxReturn', 'tillMax', 'minReturn']]

In [64]:
df

,date,maxReturn,tillMax,minReturn
228868,2017-01-01 00:00:00,0.1991,1231,-0.0072
228869,2017-01-01 00:05:00,0.2006,1230,-0.0061
228870,2017-01-01 00:10:00,0.2006,1229,-0.0061
228871,2017-01-01 00:15:00,0.1991,1228,-0.0072
228872,2017-01-01 00:20:00,0.1979,1227,-0.0083
228873,2017-01-01 00:25:00,0.1979,1226,-0.0083
228875,2017-01-01 00:30:00,0.1979,1225,-0.0083
228876,2017-01-01 00:35:00,0.2003,1224,-0.0063
228877,2017-01-01 00:40:00,0.1979,1223,-0.0083
228878,2017-01-01 00:45:00,0.1979,1222,-0.0083


In [69]:
df.query('maxReturn > 0.001').shape[0] / df.shape[0]

0.9835047062843651